# In this small project, you will (mainly) learn how to use BedTool
- .bed files are files containing "genomic regions". each line is a region here.
- BedTool is a package that helps you intersect genomic regions.

- BedTool is good at overlapping 2 or more bed files
- combining with pandas and other python scripting technique, you will be able to answer many scientific questions!


# Scientific questions:
People are looking to learn FMRP's binding specificity. They want to know what sequence/structure FMRP binds. Does FMRP bind with other proteins? Do FMRP bind UTR, CDS or intron? Does it bind m6A modifications? Does it bind G-quadruplexes. Most of those questions can be answered by overlapping many sets of data. What disease genes are bound by FMRP?

And let me show you how this is done.

# 1 Bedfile

In [1]:
# import bedtool
from pybedtools import BedTool
# read in the bed file. This bed file contains regions that are bound to FMRP
fmrp_binding_regions = BedTool('/home/hsher/projects/FMRP_UABP2L/renormalize_snake/output/FMRP_MBL_Neu_new2.peaks.normed.compressed.filtered.annotate.bed')

In [2]:
# look at the first line
# the most important columns are 0,1,2,5-> chromosome,start,end,strand. These define a "region"
print(fmrp_binding_regions[0]) # chromosome, start, end, score(not important), name(not important), strand...

chr7	5593353	5593458	38.7196019402386	5.50659033369089	+	ENSG00000075618.17	FSCN1	CDS	ENST00000382361.7:5592822:5606655:+:transcript:ENSG00000075618.17:FSCN1:protein_coding:protein_coding:feature_contains_query|ENST00000382361.7:5592822:5593768:+:exon:ENSG00000075618.17:FSCN1:protein_coding:protein_coding:feature_contains_query|ENST00000382361.7:5592936:5593768:+:CDS:ENSG00000075618.17:FSCN1:protein_coding:protein_coding:feature_contains_query



For the rest of the columns, usually (but not always) they are the ones specified in this documentation: https://genome.ucsc.edu/FAQ/FAQformat.html.

In this case above, the format is not standard format. And here are some relevant ones:
- ENSG00000075618.17: Ensembl gene ID: which gene this region is in
- FSCN1: Gene name
- CDS which region

## Task 1:
1. count how many "regions" do FMRP bind using `len()`
2. count how many unqiue genes do FMRP bind using a combination of `for` loop and `set()`
3. How many % of FMRP binding regions are in "CDS"/"UTR"/"intron". 

In [3]:
# 1-1
len(fmrp_binding_regions)

15153

In [4]:
# 1-2. Use basic python scripting...
for region in fmrp_binding_regions:
    # TODO: region[6] is gene ID
    
    pass # remove this after you fill it in

In [5]:
from collections import Counter # read about Counter. It helps you count for the number of occurence in list
# 1-3
for region in fmrp_binding_regions:
    # TODO region[8] is what you want
    
    pass # remove this after you fill it in

In [6]:
# extra.Try doing 1-2/1-3 using list comprehension

# Task 2: Overlapping

To understand if FMRP binds m6A/G4, you need some extra data. Fortunately we have other experiments probing those.

In [ ]:
import pandas as pd
# downloaded from here https://pubmed.ncbi.nlm.nih.gov/32338139/
rg4_excel = pd.read_excel('/home/hsher/projects/RG4/SupplementaryFile2.xlsx')

In [ ]:
rg4_excel.head() # this should be something that you are familiar with

In [ ]:
# the column that contains the real start/end is "Genomic intervals of rG4"
rg4_excel['start-end'] = rg4_excel['Genomic intervals of rG4'].str.split(':', expand = True)[1]
rg4_excel['start'] = rg4_excel['start-end'].str.split('-', expand = True)[0].astype(int)
rg4_excel['end'] = rg4_excel['start-end'].str.split('-', expand = True)[1].astype(int)

In [ ]:
# look at this, does it look like a bed file
rg4_excel[['chromosome', 'start', 'end', 'rG4 structural class', 'Overlapping RefSeq \ngene regions', 'strand']]

In [ ]:
# to do overlap, everything needs to become a bedtool object. You can easily to this with
rg4_bed = BedTool.from_dataframe(rg4_excel[['chromosome', 'start', 'end', 'rG4 structural class', 'Overlapping RefSeq \ngene regions', 'strand']])

In [ ]:
# Now you are about to learn about the power of bedtool.intersect

fmrp_g4 = fmrp_binding_regions.intersect(rg4_bed,
                              s = True, # why do I use s=True
                              u = True) # what does u = True mean?

In [ ]:
len(fmrp_g4) # 101 of the FMRP binding regions overlap with G4

In [ ]:
fmrp_g4[0] # these are the first one that it overlaps

tasks:
1. How many FMRP peaks DOES NOT overlap with G4?
2. How many G4 overlap with FMRP? (1 FMRP can overlap with many G4)...
3. Is this overlap sigificant. (use fisher exact test/chi-square test)
4. calculate the "jaccard distance" between G4 and FMRP using the numbers you get from above

# Task 3: To what extend does FMRP overlap with m6A modifications

In [ ]:
# Use all the technique you learned in task2 to do this
# we have m6A data in many cell lines. Here is all of them

import os
m6A_dir = '/home/hsher/projects/m6A_celltype/jackie_processed/'

path_to_m6A_beds = [os.path.join(m6A_dir,f) for f in os.listdir(m6A_dir) if f.endswith('.normed.compressed.sorted.blacklist-removed.narrowPeak')]

In [ ]:
len(path_to_m6A_beds) # a total of 60 cell lines

In [ ]:
# loop over them, check how well they overlap with FMRP. 
# Use a bar graph to display the cell line that has most m6A overlapping with FMRP

## scientific question:
1. what is m6A? 
2. Are m6A known to be different between cell lines
3. Given FMRP is CLIPped in neurons, which cell line is the most similar ones that we should compare to?
4. look at one of the m6A files. column 6 is -log 10 p-value; 7 is log 2 fold-change. What is pvalue? what is fold change? (find out at the eCLIP paper, https://www.nature.com/articles/nmeth.3810#Sec10, Normalization of eCLIP signal against SMInput.) Is all of the m6A sites confident? How will you filter them?
5. after filtering them, recheck overlapping with FMRP

In [ ]:
# 3-4 filter for significant peaks

def filter_m6A_by_pval_and_l2fc(m6A bed):
    m6A_bed = ... # TODO

    return m6A_bed.filter(lambda x: x[6]> ... and x[7]>...) #look at bedtool.filter and lambda function

In [ ]:
# 3-5 TODO imcomplete code
for bed in ...
    filtered=filter_m6A_by_pval(bed)
    filtered.intersct(fmrp.....)

# Taks 4: find overlapping proteins with FMRP
1. which encode protein overlaps most witht he encode FMRP?

In [ ]:
encode_data = pd.read_csv('/home/hsher/projects/katie/encode_data.csv')

In [ ]:
encode_data.head() # contains all encode data paths

In [ ]:
encode_data.loc[encode_data['RBP']=='FMR1'] # we have FMR1 too here

In [ ]:
# the column you want is "IDR"
fmr1_peak = BedTool(encode_data.loc[encode_data['RBP']=='FMR1', 'idr'].iloc[0])

In [ ]:
print(fmr1_peak[0])

In [ ]:
# 4-1
# now overlap FMR1 with all other proteins in encode data. Find which protein it overlaps most with
# use df.iterrows() to loop over encode_data
# make bedtool for each idr
# fmr1_peak.intersect(idr, s = True, u = True)
# calculate jaccard index for all of them, store in a dataframe.
# visualize your result using bar plots.


# 5 public data
some suggest another protein MOV10 overlaps with FMRP. 

1. google "MOV10 CLIP" to find any published CLIP data for MOV10? which species is it in? which genome coordinate is the file in (hg19/hg38/mm10).
2. Download the data. If the data is not in hg38. you need to "liftover"
3. Overlap MOV10 with FMRP and calculate jaccard index

# 6 Literature review
- based on the findings above, what do you think is the specificity of FMRP?
- What region does it bind
- Read about what people know about FMRP vs m6A/ FMRP vs G4, FMRP vs FXR protein (you should find this pair in part 4, if not, sth is wrong), FMRP vs MOV10......

# 7 Coding/bioinformatics skills you (will) leared
- you learn what is a bedfile
- you learn to use bedtool
- in fact bedtool can do a lot more than intersect/filter. This is only a small part of it. learn more reading the bedtool documentation: https://bedtools.readthedocs.io/en/latest/, "interesting usage examples"
- you learn about p-value/FC in peak calling
- you learn how to perform fisher/chi-square test
- you learn about jaccard index
- you practiced a lot of for loop/pandas/bedtools
- coding skills: list comprehension and lambda functions

